In [1]:
import logging
import os
import subprocess
import tempfile
from Bio import AlignIO

In [20]:
#!pip install biopython

In [2]:
log = logging.getLogger(__name__)

In [22]:
!pip install configobj

In [23]:
!pip install numpy

In [24]:
!pip install pytest

In [25]:
!pip install scipy

In [3]:
from tral.repeat import repeat

In [4]:
from tral import configuration

In [5]:
CONFIG_GENERAL = configuration.Configuration.instance().config
REPEAT_CONFIG = CONFIG_GENERAL["repeat"]

In [6]:
print(REPEAT_CONFIG)

{'scoreslist': ['phylo_gap01'], 'calc_score': 'False', 'calc_pvalue': 'False', 'precision': '10', 'ginsi': 'ginsi'}


## def realign_repeat
Here starts the function. The upper part will probably stay the same for any repeat aligner.

In [39]:
working_dir = tempfile.mkdtemp()
log.debug("evolvedTR: Created temp directory: %s", working_dir)

In [40]:
msa_file = os.path.join(working_dir, 'msa_temp.faa')

In [9]:
msa_file_wo_gaps = os.path.join(working_dir, 'msa_temp_wo_gaps.faa')

In [41]:
msa_file = os.path.join(working_dir, '25_13_mafft.faa')

In [42]:
print(msa_file)

/tmp/tmpb5wt73qm/25_13_mafft.faa


In [12]:
my_msa = ["MGKGYL---------------------------------------ALCSYNCKEA-INILSHLPSHHYN","TG--------------------------------------------------------------WVLCQ","IGRAYF---------------------------------------ELSEYMQAER-IFSEVRRIENYRV","EGMEIYSTTLWHLQK------------------------------DVALSVLSKDLTDMDKNSPEAWCA","AGNCFS---------LQREH-------------------------DIAIKFFQRA-IQVDPNYAYAYTL","LGHEFV--------------LTEEL--------------------DKALACFRNA-IRVNPRHYNAWYG","LGMIYY-------------------KQEKF---------------SLAEMHFQKA-LDINPQSSVLLCH","IGVVQH------------------------ALKKS----------EKALDTLNKA-IVIDPKNPLCKFH","RASVLF-----------------------------ANEKY-----KSALQELEEL-KQIVPKESLVYFL","IGKVYK----------------------------------KLGQTHLALMNFSWA-MDLDPKGAN----"]

# this example msa is from repeat_align_test.py

In [43]:
#with open(msa_file, 'w') as msa_filehandle:
#    for i, iMSA in enumerate(my_msa):
#        msa_filehandle.write('> {0}\n{1}\n'.format(i, iMSA))
        
with open(msa_file, 'w') as msa_filehandle:
    for i, iMSA in enumerate(my_msa):
        msa_filehandle.write('> seq{0}\n{1}\n'.format(i+1, iMSA))  # sequence name should not only be numbered

In [14]:
my_msa_wo_gaps = [seq.replace("-", "") for seq in my_msa] ## delete all gaps in msa

In [66]:
with open(msa_file_wo_gaps, 'w') as msa_filehandle:
    for i, iMSA in enumerate(my_msa_wo_gaps):
        msa_filehandle.write('> seq{0}\n{1}\n'.format(i, iMSA))  # sequence name should not only be numbered

In [34]:
# just to show
#with open(msa_file, 'w') as msa_filehandle:
#    for i, iMSA in enumerate(my_msa):
#        print(i,iMSA)

## if aligner =='proPIP':

In [76]:
!pip install ete3

    100% |████████████████████████████████| 10.5MB 5.3MB/s eta 0:00:01
  Running setup.py bdist_wheel for ete3 ... done
  Stored in directory: /home/lina/.cache/pip/wheels/14/90/50/5c10d965e7fb68343d9c5349cce2787ff3e38397946468d640
Successfully built ete3


In [20]:
msa_phy = os.path.join(working_dir, '25_13_mafft.phy')
AlignIO.convert(msa_file, "fasta", msa_phy, "phylip")

1

In [69]:
#input_handle = open(msa_file, "r")
msa_phy = os.path.join(working_dir, 'msa_temp.phy')
#output_handle = open(msa_phy, "w")
#alignments = AlignIO.parse(input_handle, "fasta")
#AlignIO.write(alignments, output_handle, "phylip")
#input_handle.close()
#output_handle.close()

AlignIO.convert(msa_file, "fasta", msa_phy, "phylip")

1

In [21]:
print(msa_phy)

/tmp/tmpw128cayg/25_13_mafft.phy


In [22]:
sequence_type = 'AA'

if sequence_type == 'AA':
    data_type = 'aa'
elif sequence_type == 'DNA':
    data_type = 'nt'
else:
    raise ValueError(
        'The sequence type {} is not supported.'.format(sequence_type))

In [23]:
phyml_command = [REPEAT_CONFIG['PhyML'],  
                 '--input',
                 msa_phy,
                 '--datatype',
                 data_type]

In [189]:
#phyml_command = "%s -i %s -d %s" % (REPEAT_CONFIG['PhyML'], msa_phy, data_type)

In [24]:
phyml_command

['PhyML-3.1_linux64',
 '--input',
 '/tmp/tmpw128cayg/25_13_mafft.phy',
 '--datatype',
 'aa']

In [197]:
# p = subprocess.Popen(phyml_command.split())
# this command split would create an error if there are any white spaces
# in the user path. good practise to not use something like this

In [25]:
p = subprocess.Popen(phyml_command)

In [26]:
tree_file = msa_phy + "_phyml_tree.txt"
with open (tree_file,"r") as tree:
    tree = tree.read()

In [27]:
print(tree)

(1737_2:0.87330411,1737_3:0.31435158,((((1737_10:0.34801093,1737_0:0.57030369)0.658000:0.09102283,1737_8:0.29216895)0.748000:0.09202513,1737_6:1.24928901)0.000000:0.00000670,(((1737_5:0.83280688,(1737_1:0.84916076,(1737_9:1.50975292,1737_11:1.56771714)0.000000:0.00007046)0.000000:0.00003275)0.000000:0.00003069,1737_7:0.26156496)0.726000:0.15987827,1737_4:0.92209976)0.000000:0.00002967)0.898000:0.54880003);



In [28]:
from ete3 import Tree
t = Tree(tree)
print(t)


   /-1737_2
  |
  |--1737_3
  |
  |            /-1737_10
  |         /-|
--|      /-|   \-1737_0
  |     |  |
  |   /-|   \-1737_8
  |  |  |
  |  |   \-1737_6
  |  |
  |  |         /-1737_5
   \-|      /-|
     |     |  |   /-1737_1
     |     |   \-|
     |   /-|     |   /-1737_9
     |  |  |      \-|
     |  |  |         \-1737_11
      \-|  |
        |   \-1737_7
        |
         \-1737_4


In [29]:
t.resolve_polytomy()
print(t)


      /-1737_3
     |
     |            /-1737_10
     |         /-|
     |      /-|   \-1737_0
   /-|     |  |
  |  |   /-|   \-1737_8
  |  |  |  |
  |  |  |   \-1737_6
  |  |  |
  |  |  |         /-1737_5
  |   \-|      /-|
  |     |     |  |   /-1737_1
--|     |     |   \-|
  |     |   /-|     |   /-1737_9
  |     |  |  |      \-|
  |     |  |  |         \-1737_11
  |      \-|  |
  |        |   \-1737_7
  |        |
  |         \-1737_4
  |
   \-1737_2


In [38]:
new_tree = t.write(format=1)

In [39]:
print(new_tree)

((1737_3:0.314352,((((1737_10:0.348011,1737_0:0.570304):0.0910228,1737_8:0.292169):0.0920251,1737_6:1.24929):6.7e-06,(((1737_5:0.832807,(1737_1:0.849161,(1737_9:1.50975,1737_11:1.56772):7.046e-05):3.275e-05):3.069e-05,1737_7:0.261565):0.159878,1737_4:0.9221):2.967e-05):0.5488):0,1737_2:0.873304);


In [40]:
xx = new_tree.replace(":0,", ":0.0000001") ## replace branchlength 0 with 0.0000001
#new_tree.replace(":0)", ":0.0000001)")

In [41]:
print(xx)

((1737_3:0.314352,((((1737_10:0.348011,1737_0:0.570304):0.0910228,1737_8:0.292169):0.0920251,1737_6:1.24929):6.7e-06,(((1737_5:0.832807,(1737_1:0.849161,(1737_9:1.50975,1737_11:1.56772):7.046e-05):3.275e-05):3.069e-05,1737_7:0.261565):0.159878,1737_4:0.9221):2.967e-05):0.5488):0.0000001,1737_2:0.873304);


In [125]:
changed_tree = Tree("((seq1:1.23905,(seq9:0.691308,((seq8:2.01072,seq7:0.73598):0.442514,((seq4:1.05147,(seq5:0.883349,seq3:1.39482):0.254853):0.151611,(seq0:1.1512,seq2:2.03936):1.18825):0.242604):0.416472):0.550751):0.0001,seq6:0.0926462);")

In [126]:
print(changed_tree)


      /-seq1
     |
   /-|   /-seq9
  |  |  |
  |  |  |      /-seq8
  |   \-|   /-|
  |     |  |   \-seq7
  |     |  |
  |      \-|      /-seq4
  |        |   /-|
--|        |  |  |   /-seq5
  |        |  |   \-|
  |         \-|      \-seq3
  |           |
  |           |   /-seq0
  |            \-|
  |               \-seq2
  |
   \-seq6


In [199]:
./miniJATI alphabet=DNA alignment=true input.sequence.file=notaligned_real_shh300_nt.fa input.sequence.sites_to_use=all model="PIP(model=GTR(),lambda=0.1,mu=0.2)" rate_distribution="Constant()" init.tree=user input.tree.file=real_real_shh300_nt.tree.nwk optimization="ND-BFGS(derivatives=BFGS)" optimization.ignore_parameters=BrLen,Model optimization.max_number_f_eval=1000 optimization.tolerance=0.001 optimization.final=bfgs optimization.topology=false output.msa.file=out_alignmment.fa output.tree.file=out_tree.nwk output.infos=out_info.log output.estimates=.out_estimates.log

## if aligner =='mafft':
Start with the mafft specific input.

How to get stdout and stderr using Python's subprocess module:
https://www.saltycrane.com/blog/2008/09/how-get-stdout-and-stderr-using-python-subprocess-module/

In [44]:
# I have to define an msa file. An example file is given here.
p = subprocess.Popen([REPEAT_CONFIG['ginsi'],
                      "--anysymbol",
                      "--quiet",
                      "/home/lina/SynologyDrive/TRAL_Masterthesis/Notes/alignment/repeat_align/example_sequences/msa.faa"], 
                     stdout=subprocess.PIPE)

In [25]:
# create an new test msa:
p = subprocess.Popen([REPEAT_CONFIG['ginsi'],
                      "--anysymbol",
                      "--quiet",
                      "/home/lina/SynologyDrive/TRAL_Masterthesis/program_testing/miniJati/Testfiles/25_13.faa"])

I will look at the following line a little bit in detail to understand the whole function:

In [ ]:
# mafft_output = [line.decode('utaf8').rstrip() for line in p.stdout]

In [29]:
print(p)

This methods only work once! So the ginsi output has to be in stdout to read it or to decode it!

In [31]:
p.stdout.read()

b''

In [27]:
[line for line in p.stdout]

[]

In [19]:
encoded_output = [line.decode('utf8') for line in p.stdout]
print(encoded_output)

[]


In [ ]:
encoded_output

In [20]:
p.stdout

<_io.BufferedReader name=44>

The method rstrip() returns a copy of the string in which all chars have been stripped from the end of the string (default whitespace characters).

In [45]:
mafft_output = [line.decode('utf8').rstrip() for line in p.stdout]

In [21]:
mafft_output = [line.rstrip() for line in encoded_output] # here it strips for example the \n!

In [46]:
mafft_output

['> 0',
 'MGKGYLALC----SYNC-----KEAI-NILSHLPSHHYN',
 '> 1',
 'TGWVL--------------------------------CQ',
 '> 2',
 'IGRAYFELS----EYMQAERIFSEVR-RIEN-----YRV',
 '> 3',
 'EGMEIYSTTLWHLQKDVALSVLSKDLTDMDKNSPEAWCA',
 '> 4',
 'AGNCFSLQR----EHDIAIKFFQRAI-QVDPNYAYAYTL',
 '> 5',
 'LGHEFVLTE----ELDKALACFRNAI-RVNPRHYNAWYG',
 '> 6',
 'LGMIYYKQE----KFSLAEMHFQKAL-DINPQSSVLLCH',
 '> 7',
 'IGVVQHALK----KSEKALDTLNKAI-VIDPKNPLCKFH',
 '> 8',
 'RASVLFANE----KYKSALQELEELK-QIVPKESLVYFL',
 '> 9',
 'IGKVYKKLG----QTHLALMNFSWAM-DLDPKGA----N']

In [53]:
type(mafft_output)

list

In [54]:
for iLine in mafft_output:
    print(iLine)

> 0
MGKGYLALC----SYNC-----KEAI-NILSHLPSHHYN
> 1
TGWVL--------------------------------CQ
> 2
IGRAYFELS----EYMQAERIFSEVR-RIEN-----YRV
> 3
EGMEIYSTTLWHLQKDVALSVLSKDLTDMDKNSPEAWCA
> 4
AGNCFSLQR----EHDIAIKFFQRAI-QVDPNYAYAYTL
> 5
LGHEFVLTE----ELDKALACFRNAI-RVNPRHYNAWYG
> 6
LGMIYYKQE----KFSLAEMHFQKAL-DINPQSSVLLCH
> 7
IGVVQHALK----KSEKALDTLNKAI-VIDPKNPLCKFH
> 8
RASVLFANE----KYKSALQELEELK-QIVPKESLVYFL
> 9
IGKVYKKLG----QTHLALMNFSWAM-DLDPKGA----N


In [70]:
# just to test how it works
msa = []
for iLine in mafft_output:
    if iLine[0] == '>':
        print(iLine)
    else:
        msa.append('') # just to add a new string to the list
        msa[-1] += iLine # a negative index means you read from the right, [-1] is the last element!
print(msa)

> 0
> 1
> 2
> 3
> 4
> 5
> 6
> 7
> 8
> 9
['MGKGYLALC----SYNC-----KEAI-NILSHLPSHHYN', 'TGWVL--------------------------------CQ', 'IGRAYFELS----EYMQAERIFSEVR-RIEN-----YRV', 'EGMEIYSTTLWHLQKDVALSVLSKDLTDMDKNSPEAWCA', 'AGNCFSLQR----EHDIAIKFFQRAI-QVDPNYAYAYTL', 'LGHEFVLTE----ELDKALACFRNAI-RVNPRHYNAWYG', 'LGMIYYKQE----KFSLAEMHFQKAL-DINPQSSVLLCH', 'IGVVQHALK----KSEKALDTLNKAI-VIDPKNPLCKFH', 'RASVLFANE----KYKSALQELEELK-QIVPKESLVYFL', 'IGKVYKKLG----QTHLALMNFSWAM-DLDPKGA----N']


Here I found more easy way to create a list for msa:

In [47]:
msa = [iLine for iLine in mafft_output if iLine[0] != '>']

# first simplyfication

# msa = []
# for iLine in mafft_output:
#     if iLine[0] != '>':
#         msa.append(iLine)
        
# Original in TRAL:

# msa = []
# for iLine in mafft_output:
#     if iLine[0] == '>':
#         msa.append('')
#     else:
#         msa[-1] += iLine

In [49]:
msa = [i for i in msa if i != '']

In [50]:
print(msa)

['MGKGYLALC----SYNC-----KEAI-NILSHLPSHHYN', 'TGWVL--------------------------------CQ', 'IGRAYFELS----EYMQAERIFSEVR-RIEN-----YRV', 'EGMEIYSTTLWHLQKDVALSVLSKDLTDMDKNSPEAWCA', 'AGNCFSLQR----EHDIAIKFFQRAI-QVDPNYAYAYTL', 'LGHEFVLTE----ELDKALACFRNAI-RVNPRHYNAWYG', 'LGMIYYKQE----KFSLAEMHFQKAL-DINPQSSVLLCH', 'IGVVQHALK----KSEKALDTLNKAI-VIDPKNPLCKFH', 'RASVLFANE----KYKSALQELEELK-QIVPKESLVYFL', 'IGKVYKKLG----QTHLALMNFSWAM-DLDPKGA----N']


In [81]:
print('\n'.join(msa))

MGKGYLALC----SYNC-----KEAI-NILSHLPSHHYN
TGWVL--------------------------------CQ
IGRAYFELS----EYMQAERIFSEVR-RIEN-----YRV
EGMEIYSTTLWHLQKDVALSVLSKDLTDMDKNSPEAWCA
AGNCFSLQR----EHDIAIKFFQRAI-QVDPNYAYAYTL
LGHEFVLTE----ELDKALACFRNAI-RVNPRHYNAWYG
LGMIYYKQE----KFSLAEMHFQKAL-DINPQSSVLLCH
IGVVQHALK----KSEKALDTLNKAI-VIDPKNPLCKFH
RASVLFANE----KYKSALQELEELK-QIVPKESLVYFL
IGKVYKKLG----QTHLALMNFSWAM-DLDPKGA----N
